### 載入套件 和定義函式

dcard 標籤要多注意，因為很常被更換

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
import urllib.request as req
import numpy as np
from scipy.linalg import norm
import pandas as pd
import warnings

def tfidf_similarity(s1, s2):
    def add_space(s):
        return ' '.join(list(s))
    # 將字中間加入空格
    s1, s2 = add_space(s1), add_space(s2)
    # 轉化為TF矩陣
    cv = TfidfVectorizer(tokenizer=lambda s: s.split())
    corpus = [s1, s2]
    vectors = cv.fit_transform(corpus).toarray()
    
    # 計算TF係數
    fiend = np.dot(vectors[0], vectors[1]) / (norm(vectors[0]) * norm(vectors[1])) 
    np.seterr(invalid='ignore') # 當計算結果為無意義(分母為0)，忽略此警告
    return fiend
    
def stopwordslist(filepath):
    stopwords = [line.strip() for line in open(filepath, 'r', encoding='utf-8').readlines()]
    return stopwords

def movestopwords(sentence):
    stopwords = stopwordslist('stopword.txt')  # 這裏加載停用詞的路徑
    outstr = ''
    for word in sentence:           
        if word not in stopwords:  
            if word != '\t'and'\n':
                outstr += word
    return outstr

def getWord(url):
    # 建立一個Request 物件，附加Request Headers 的資訊
    request = req.Request(url, headers={
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.60 Safari/537.36"
    })
    with req.urlopen(request) as response:
        data = response.read().decode("utf-8")
    # 「解析」原始碼，取得每篇文章的問題
    # utf-8(比較省空間)有部分的漢字不能轉換所以要用GB18030編碼

    import bs4
    # 讓beautifulSoup協助我們解析HTML格式文件
    root = bs4.BeautifulSoup(data, "html.parser")
    titles = root.find("div", class_="sc-ba53eaa8-0 hKkUKs")  # 用列表顯示全部爬蟲下來的標題

    for title in titles:
        result = title.text.strip().replace('\n', '').replace(' ', '')
    return result
    # titles代表div標籤
    # 尋找class = "title" 的div 標籤，因為class是保留字，所以寫成class_
    # root 代表整個網頁、title是網頁標籤也是網頁標題
    # cls 是清空終端機(terminal)
    # mode = "a"是以附加的方式打開並寫入文件，因為mode = "w"會將檔案清空在寫入，mode="a"不會清空



### 抓取檔案

In [16]:
train = pd.read_csv('done_2021-08to12.csv')
# article = getWord('https://www.dcard.tw/f/relationship/p/238632575')
# article = getWord('https://www.dcard.tw/f/talk/p/239984330')
# article = getWord('https://www.dcard.tw/f/relationship/p/240103776')
article = getWord('https://www.dcard.tw/f/talk/p/239983442')

In [14]:
train

,singer,name,lyrics,moodCat
0,金玟岐,你註定會遇見我,"['我愛上', '這塵世紛紛擾擾的相遇', '愛上', '重複俗氣又沉重的春天', '愛上'...",愛
1,張杰,看吶看,"['你那彩虹般的微笑', '撞擊我加快的心跳', '融化我過往的驕傲', '我變得渺小', ...",愛
2,"你好,我是接體員 原聲帶","你好,我是你的","['你是幹什麼的?', '我是幹什麼', '你是幹什麼的?', '我是幹', '你是幹什麼的...",哀
3,"你好,我是接體員 原聲帶",彼邊,"['後面的呀~張緊緊喔!', '彼邊是行不知路的世界,人講天堂和地獄的所在。', '整天攏有...",懼
4,"你好,我是接體員 原聲帶",你們都來看我嗎 ?,"['selfie selfie selfie selfie', '你一定會愛我我也會愛...",喜
...,...,...,...,...
4223,盧廣仲,明年(Let Go),"['今天的心比昨天的心忙', '今天的我比昨天的我胖', '今天的黑夜比昨天還長', '今天...",喜
4224,盧廣仲,英雄(Heroes),"['太長的一段旅程 堅持到底為什麼', '望著一片大海 還是像個孩子', '浪花衝垮了勇...",愛
4225,十七,忘了呼吸,"['半夜手機的R&B', '我想著你', '無法睡去', '在我心底你是唯一', '忘了呼吸...",愛
4226,唐詩,逍遙遊,"['一輪月色霜滿天 誰臨幸人間', '孤煙落日全都看遍', '青川走馬遊蕩山水間', '蝶飛...",哀


### 計算配適度

In [17]:
lyrics=train['lyrics']
i=0
num=0
highpri=0
for text in lyrics:
    text=movestopwords(text)
    text=text.replace(' ','')
    text=text.replace(',','，')
    if tfidf_similarity(text, article)>highpri:
        highpri=tfidf_similarity(text, article)
        num=i
    i+=1
print('配適度:',highpri,'作者:',train.singer.iloc[num],'歌名:',train.name.iloc[num], '情緒:',train.moodCat.iloc[num])


配適度: 0 作者: 金玟岐 歌名: 你註定會遇見我 情緒: 愛
